# Installs

In [1]:
pip install chroma-haystack

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install transformers[torch,sentencepiece]

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install "sentence-transformers>=3.0.0"

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install haystack-ai datasets ollama-haystack gradio

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install ollama-haystack

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install haystack-ai datasets dspy-ai sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.5/272.5 kB 7.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 64.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 67.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.9/603.9 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 9.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [42]:
pip install hf_xet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.5/25.5 MB 70.4 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


# Imports

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
from haystack import Document, Pipeline
from haystack.components.builders import PromptBuilder
from haystack_integrations.components.generators.ollama import OllamaGenerator
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

In [3]:
from haystack_integrations.document_stores.chroma import ChromaDocumentStore
from haystack_integrations.components.retrievers.chroma import ChromaEmbeddingRetriever, ChromaQueryTextRetriever

In [4]:
from haystack.components.embedders import HuggingFaceAPIDocumentEmbedder
from haystack_integrations.components.embedders.ollama import OllamaDocumentEmbedder, OllamaTextEmbedder
from haystack.utils import Secret

from haystack.components.embedders import HuggingFaceAPITextEmbedder
from haystack.components.preprocessors import DocumentSplitter

In [5]:
import dspy
from dspy import Signature, InputField, OutputField, Module, Retrieve, ChainOfThought, Prediction, Predict

In [6]:
from dspy import BootstrapFewShot
from dspy.evaluate.metrics import answer_exact_match, answer_passage_match

from dspy.teleprompt import BootstrapFewShot, MIPROv2

In [7]:
from haystack.components.evaluators import SASEvaluator

In [8]:
import shutil

In [9]:
from haystack.components.builders import AnswerBuilder

In [10]:
from dspy.evaluate.evaluate import Evaluate

In [36]:
import json

# Dataset

In [11]:
df = pd.read_csv('dataset_code.csv')
#df = df[df['vulnerability'].str.contains('reentrancy')].reset_index(drop=True)
#df = df.head(100)
df

,name,vulnerability,line,code
0,0x000000000019fff0e5b945e90ee1e606aa22c6c2_non...,access_control,"1005, 1006, 1010, 1019, 1036, 1045, 1049, 1055...",\n\n\n\npragma solidity 0.5.16;\n\n\ninterface...
1,0x00000624264253944a702b8b812ae7bbb46af371_int...,integer_overflow,179,\n\n\n\npragma solidity =0.5.16;\n\ninterface ...
2,0x00022837E66Fdd4bbFe63859d7dc12651738D329_int...,integer_overflow,"556, 558, 564, 566",\n\n\n\n\n\npragma solidity 0.5.16;\n\n\n\n\nl...
3,0x000c100050e98c91f9114fa5dd75ce6869bf4f53_int...,integer_overflow,"233, 253, 272, 899",\n\n\n\n\n\n\n\n\n\npragma solidity ^0.5.6;\n\...
4,0x0051aa458da83d98610742851255b1b64f2b70fa_int...,integer_overflow,"272, 274, 276, 278, 438, 500, 542",\n\n\n\npragma solidity >=0.6.8;\n\n\n\n\ncont...
...,...,...,...,...
2043,xMPH.sol,unchecked_low_level_calls,"L178, L196, L212",\npragma solidity 0.8.3;\n\nimport {ERC20} fro...
2044,yToken.sol,reentrancy,L187,\npragma solidity ^0.6.12;\npragma experimenta...
2045,yToken.sol,denial_of_service,"L108 - L110, L156 - L159",\npragma solidity ^0.6.12;\npragma experimenta...
2046,yWETH.sol,reentrancy,"L144 - L162, L90 - L104","pragma solidity ^0.5.16;\n\nimport ""@openzeppe..."


In [12]:
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42, shuffle=True)
print(len(train_df),len(test_df))

1843 205


# Pipeline

In [13]:
documents = [
    Document(
        content=row['code'],
        meta = {
            "vulnerability": row['vulnerability'],
            "line": row["line"]
            }
    )
    for index, row in train_df.iterrows()
]

In [14]:
document_embedder = OllamaDocumentEmbedder(model="starcoder2:3b", url = "http://localhost:11434")
document_store = ChromaDocumentStore()

result = document_embedder.run(documents)
embedded_documents = result["documents"]
document_store.write_documents(embedded_documents)

Calculating embeddings: 100%|██████████| 58/58 [39:36<00:00, 40.98s/it]


1843

In [15]:
embedding_retriever = ChromaEmbeddingRetriever(document_store=document_store)

llm = OllamaGenerator(model="deepseek-coder-v2:16b",
                            url = "http://localhost:11434",
                            generation_kwargs={
                              #"num_predict": 300,
                              "temperature": 0.7,
                              "timeout": 5000,
                              "num_ctx": 16384
                              })

text_embedder = OllamaTextEmbedder(model="starcoder2:3b", url = "http://localhost:11434")

In [16]:
lm = dspy.LM(
    model='ollama_chat/deepseek-coder-v2:16b',
    api_base='http://localhost:11434',
    api_key='',
    config={
        "num_ctx": 16384  # ou até 163840
    }
)
dspy.settings.configure(lm=lm)

In [50]:
class VulnSig(Signature):
    """Identify if the Solidity smart contract has any type of reentrancy vulnerability with short answers."""
    context = InputField(desc="Relevant retrieved context to help with the analysis.")
    question = InputField(desc="The Solidity smart contract code to analyze.")
    #answer = OutputField(desc="Detected vulnerability type.")
    json_output = OutputField(desc="JSON object with key-value pairs describing the detected vulnerability. Example: {'vulnerability': 'reentrancy'}")

class RAG(Module):
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.ChainOfThought(VulnSig)

    def retrieve(self, question):
        # faço embedding da entrada
        embedding = text_embedder.run(question)["embedding"]
        # procuro pelo embedding
        results = embedding_retriever.run(query_embedding=embedding)
        passages = [res.content for res in results["documents"]]
        return Prediction(passages=passages)

    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, 
                               #answer=prediction.answer, 
                               json_output=prediction.json_output)

uncompiled_rag = RAG()

In [51]:
code_example = """
// SPDX-License-Identifier: MIT
pragma solidity 0.8.29;

contract EtherStore {
    mapping(address => uint256) public balances;

    function deposit() public payable {
        balances[msg.sender] += msg.value;
    }

    function withdraw() public {
        uint256 bal = balances[msg.sender];
        require(bal > 0);

        (bool sent,) = msg.sender.call{value: bal}("");
        require(sent, "Failed to send Ether");

        balances[msg.sender] = 0;
    }

    function getBalance() public view returns (uint256) {
        return address(this).balance;
    }
}

"""

In [53]:
response = uncompiled_rag(question=code_example)
json_response = json.loads(response.json_output)
print(json_response["answer"])

InternalError: Error getting collection: Database error: error returned from database: (code: 1) no such table: collections

In [46]:
trainset, devset = [], []

for i, row in df.iterrows():
    example = dspy.Example(
        question = row["code"],  # código é a entrada
        answer = row["vulnerability"]  # vulnerabilidade e linha são a saída
    ).with_inputs("question")  # determina o campo de entrada
    
    # 30 primeiras linhas como exemplo
    if i < 10:
        trainset.append(example)
    elif i < 30:
        devset.append(example)
    else:
        break

In [ ]:
# sentence-transformers/paraphrase-multilingual-mpnet-base-v2
sas_evaluator = SASEvaluator()
sas_evaluator.warm_up()

def mixed_metric(example, pred, trace=None):
    semantic_similarity = sas_evaluator.run(ground_truth_answers=[example.answer], predicted_answers=[pred.answer])["score"]

    n_words=len(pred.answer.split())
    long_answer_penalty=0
    if 20<n_words<40:
      long_answer_penalty = 0.025 * (n_words - 20)
    elif n_words>=40:
      long_answer_penalty = 0.5

    #return semantic_similarity - long_answer_penalty
    return semantic_similarity

In [27]:
from dspy.evaluate.evaluate import Evaluate

evaluate = Evaluate(
    metric=mixed_metric, devset=devset, num_threads=1, display_progress=True, display_table=5
)
evaluate(uncompiled_rag)

Average Metric: 2.23 / 20 (11.2%): 100%|██████████| 20/20 [01:42<00:00,  5.12s/it]

2025/06/03 12:20:58 INFO dspy.evaluate.evaluate: Average Metric: 2.2308204881846905 / 20 (11.2%)


,question,example_answer,context,pred_answer,json_output,mixed_metric
0,pragma solidity >=0.5.4 <0.6.0; interface tokenRecipient { functio...,access_control,['\n\n\n\npragma solidity >=0.6.8;\n\n\n\n\ncontract ownerShip\n{\...,,{},✔️ [0.108]
1,pragma solidity ^0.4.19; contract PERSONAL_BANK { mapping (address...,reentrancy,['\n\n\n\npragma solidity >=0.6.8;\n\n\n\n\ncontract ownerShip\n{\...,,,✔️ [0.294]
2,pragma solidity 0.5.17; pragma experimental ABIEncoderV2; interfac...,integer_overflow,['\n\n\n\npragma solidity >=0.6.8;\n\n\n\n\ncontract ownerShip\n{\...,,,✔️ [0.063]
3,pragma solidity ^0.5.1; interface ERC20 { function balanceOf(addre...,integer_overflow,['\n\n\n\npragma solidity >=0.6.8;\n\n\n\n\ncontract ownerShip\n{\...,SafeMath,,✔️ [0.167]
4,pragma solidity ^0.6.12; interface IERC20 { function totalSupply()...,integer_overflow,['\n\n\n\npragma solidity >=0.6.8;\n\n\n\n\ncontract ownerShip\n{\...,,{},✔️ [0.063]


np.float64(11.15)

In [30]:
#optimizer = BootstrapFewShot(metric=mixed_metric)
#compiled_rag = optimizer.compile(RAG(), trainset=trainset)

optimizer = MIPROv2(
    metric=mixed_metric,
    auto="light"
)

# Otimizar o programa sem exemplos few-shot
compiled_rag = optimizer.compile(
    RAG(),
    trainset=trainset,
    max_bootstrapped_demos=0,
    max_labeled_demos=0,
    requires_permission_to_run=False
)

2025/05/30 13:43:14 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 10
minibatch: False
num_fewshot_candidates: 6
num_instruct_candidates: 3
valset size: 8

2025/05/30 13:43:14 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/05/30 13:43:14 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used for informing instruction proposal.

2025/05/30 13:43:14 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=6 sets of demonstrations...


Bootstrapping set 1/6
Bootstrapping set 2/6


100%|██████████| 2/2 [00:09<00:00,  4.57s/it]


Bootstrapped 2 full traces after 1 examples for up to 1 rounds, amounting to 2 attempts.
Bootstrapping set 3/6


100%|██████████| 2/2 [00:03<00:00,  1.59s/it]


...

2025/05/30 13:50:41 INFO dspy.evaluate.evaluate: Average Metric: 1.2167237028479576 / 8 (15.2%)
2025/05/30 13:50:41 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 15.21 with parameters ['Predictor 0: Instruction 1', 'Predictor 0: Few-Shot Set 4'].
2025/05/30 13:50:41 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [2.31, 20.04, 0.82, 9.43, 9.94, 7.71, 6.11, 12.23, 15.21]
2025/05/30 13:50:41 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 20.04
2025/05/30 13:50:41 INFO dspy.teleprompt.mipro_optimizer_v2: ========================


2025/05/30 13:50:41 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 10 / 10 =====



Average Metric: 0.72 / 8 (9.0%): 100%|██████████| 8/8 [00:41<00:00,  5.17s/it] 

2025/05/30 13:51:23 INFO dspy.evaluate.evaluate: Average Metric: 0.7174215093255043 / 8 (9.0%)
2025/05/30 13:51:23 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 8.97 with parameters ['Predictor 0: Instruction 2', 'Predictor 0: Few-Shot Set 5'].
2025/05/30 13:51:23 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [2.31, 20.04, 0.82, 9.43, 9.94, 7.71, 6.11, 12.23, 15.21, 8.97]
2025/05/30 13:51:23 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 20.04
2025/05/30 13:51:23 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/05/30 13:51:23 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 11 / 10 =====



Average Metric: 0.79 / 8 (9.9%): 100%|██████████| 8/8 [00:44<00:00,  5.51s/it] 

2025/05/30 13:52:07 INFO dspy.evaluate.evaluate: Average Metric: 0.7915322035551071 / 8 (9.9%)
2025/05/30 13:52:07 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 9.89 with parameters ['Predictor 0: Instruction 1', 'Predictor 0: Few-Shot Set 3'].
2025/05/30 13:52:07 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [2.31, 20.04, 0.82, 9.43, 9.94, 7.71, 6.11, 12.23, 15.21, 8.97, 9.89]
2025/05/30 13:52:07 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 20.04
2025/05/30 13:52:07 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/05/30 13:52:07 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 20.04!


In [31]:
evaluate = Evaluate(
    metric=mixed_metric, devset=devset, num_threads=1, display_progress=True, display_table=5
)
evaluate(compiled_rag)

Average Metric: 2.05 / 20 (10.3%): 100%|██████████| 20/20 [03:46<00:00, 11.33s/it]

2025/05/30 13:58:18 INFO dspy.evaluate.evaluate: Average Metric: 2.0542424701154234 / 20 (10.3%)


,question,example_answer,context,pred_answer,json_output,mixed_metric
0,pragma solidity >=0.5.4 <0.6.0; interface tokenRecipient { functio...,access_control,['\n\n\n\npragma solidity >=0.6.8;\n\n\n\n\ncontract ownerShip\n{\...,,,✔️ [0.108]
1,pragma solidity ^0.4.19; contract PERSONAL_BANK { mapping (address...,reentrancy,['\n\n\n\npragma solidity >=0.6.8;\n\n\n\n\ncontract ownerShip\n{\...,,,✔️ [0.294]
2,pragma solidity 0.5.17; pragma experimental ABIEncoderV2; interfac...,integer_overflow,['\n\n\n\npragma solidity >=0.6.8;\n\n\n\n\ncontract ownerShip\n{\...,,,✔️ [0.063]
3,pragma solidity ^0.5.1; interface ERC20 { function balanceOf(addre...,integer_overflow,['\n\n\n\npragma solidity >=0.6.8;\n\n\n\n\ncontract ownerShip\n{\...,Solidity,,✔️ [0.157]
4,pragma solidity ^0.6.12; interface IERC20 { function totalSupply()...,integer_overflow,['\n\n\n\npragma solidity >=0.6.8;\n\n\n\n\ncontract ownerShip\n{\...,,{},✔️ [0.063]


np.float64(10.27)

In [32]:
lm.inspect_history(n=1)





[2025-05-30T13:58:18.408266]

System message:

Your input fields are:
1. `context` (str): Relevant retrieved context to help with the analysis.
2. `question` (str): The Solidity smart contract code to analyze.
Your output fields are:
1. `reasoning` (str)
2. `answer` (str): Detected vulnerability type.
3. `json_output` (str): key-value pair of the vulnerability
All interactions will be structured in the following way, with the appropriate values filled in.

Inputs will have the following structure:

[[ ## context ## ]]
{context}

[[ ## question ## ]]
{question}

Outputs will be a JSON object with the following fields.

{
  "reasoning": "{reasoning}",
  "answer": "{answer}",
  "json_output": "{json_output}"
}
In adhering to this structure, your objective is: 
        Given the fields `context`, `question`, produce the fields `reasoning`, `answer`, `json_output`.


User message:

[[ ## context ## ]]
[1] «««
    
    
    
    
    pragma solidity >=0.6.8;
    
    
    
    
    contr

In [25]:
static_prompt = """
Your input fields are:
1. `question` (str): The Solidity smart contract code to analyze.
Your output fields are:
1. `reasoning` (str)
2. `answer` (str): Detected vulnerability type and line numbers that are vulnerable.
All interactions will be structured in the following way, with the appropriate values filled in.

Inputs will have the following structure:

[[ ## question ## ]]
{question}

Outputs will be a JSON object with the following fields.

{
  "reasoning": "{reasoning}",
  "answer": "{answer}"
}
In adhering to this structure, your objective is: 
        Identify if the Solidity smart contract has any type of reentrancy vulnerability and which line it is present in with short answers.


Respond with a JSON object in the following order of fields: `reasoning`, then `answer`.

"""

In [26]:
prompt_baseline = static_prompt + """
Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}
<|eot_id|>

Smart Contract Code: {{question}}
"""

prompt_builder = PromptBuilder(
    template=prompt_baseline,
    required_variables=["question", "documents"]
)

In [27]:
rag_pipeline = Pipeline()

rag_pipeline.add_component("text_embedder", instance = text_embedder)
rag_pipeline.add_component("embedding_retriever", embedding_retriever)
rag_pipeline.add_component("prompt_builder", prompt_builder)
rag_pipeline.add_component("generator", instance = llm)

In [28]:
rag_pipeline.connect("text_embedder.embedding", "embedding_retriever.query_embedding")
rag_pipeline.connect("embedding_retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "generator")


🚅 Components
  - text_embedder: OllamaTextEmbedder
  - embedding_retriever: ChromaEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - generator: OllamaGenerator
🛤️ Connections
  - text_embedder.embedding -> embedding_retriever.query_embedding (List[float])
  - embedding_retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> generator.prompt (str)

# Execução

In [29]:
import time
count = 0
total_time = 0

for index, row in test_df.iterrows():
    start_time = time.time()  # Início da contagem

    resposta = rag_pipeline.run({
        "text_embedder": {"text": row["code"]},
        "prompt_builder": {"question": row["code"]}
    })

    end_time = time.time()  # Fim da contagem
    count+=1

    duration = end_time - start_time
    total_time += duration
    print('\n----------------------------------------------')
    print(resposta["generator"]["replies"][0])
    print(f"Tempo de resposta: {duration:.2f} segundos\n")
    print('----------------------------------------------\n')

print(f"Tempo médio: {(total_time/count):.2f} segundos\n")


----------------------------------------------
It seems like you've provided a contract for an NFT (Non-Fungible Token), the Golff token, as well as some additional details about this specific blockchain and smart contracts. This is quite comprehensive but there are still several things I would recommend reviewing or expanding upon in your code:

1) Governance Mechanism - The current implementation does not include a governance mechanism for changing parameters such as minimum share value, controller address etc., which could potentially lead to changes unintentionally. You may want to consider adding some sort of role-based access control (RBAC), or even more advanced mechanisms like the Proposal Pattern that allows people who are not fully knowledgeable on a topic change their minds and propose an alteration, before being finalized by majority vote if necessary.
   
2) Reentrancy - The contract has some functions which call other public/internal smart contracts methods (delegated tr

PipelineRuntimeError: The following component failed to run:
Component name: 'generator'
Component type: 'OllamaGenerator'
Error: timed out